In [1]:
# !pip3 install malaya-speech -U --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import malaya_speech
import json

malaya_speech.__version__

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

'1.2.6.1'

In [4]:
from glob import glob
malay = sorted(glob('/home/ubuntu/wav2vec2/malay-test/*.wav'), key = lambda x: int(x.split('/')[-1].replace('.wav', '')))
singlish = sorted(glob('/home/ubuntu/wav2vec2/singlish-test/*.wav'), key = lambda x: int(x.split('/')[-1].replace('.wav', '')))
len(malay), len(singlish)

(765, 3579)

In [5]:
with open('/home/ubuntu/wav2vec2/malay-test.json') as fopen:
    malay_label = json.load(fopen)
with open('/home/ubuntu/wav2vec2/singlish-test.json') as fopen:
    singlish_label = json.load(fopen)
    
len(malay_label), len(singlish_label)

(765, 3579)

In [6]:
from sklearn.utils import shuffle

audio = malay + singlish
labels = malay_label + singlish_label
audio, labels = shuffle(audio, labels)
test_set = list(zip(audio, labels))
test_set[:10]

[('/home/ubuntu/wav2vec2/malay-test/661.wav',
  'kenapa orang orang muslim tidak menonjol melawan spanyol'),
 ('/home/ubuntu/wav2vec2/singlish-test/2831.wav',
  'and as if that was mundane enough boyd decided to pick it up'),
 ('/home/ubuntu/wav2vec2/singlish-test/2240.wav',
  'her husband disappeared leaving her and her two daughters to fend for themselves'),
 ('/home/ubuntu/wav2vec2/singlish-test/1530.wav', 'her head wasnt in it'),
 ('/home/ubuntu/wav2vec2/singlish-test/954.wav',
  'its all about passing the flame from one generation to another'),
 ('/home/ubuntu/wav2vec2/singlish-test/3465.wav',
  'air conditioning is an important part of our singaporean life'),
 ('/home/ubuntu/wav2vec2/singlish-test/2433.wav',
  'bank of china now offers funds transfer services through paynow'),
 ('/home/ubuntu/wav2vec2/singlish-test/165.wav',
  'can athletes be given more help'),
 ('/home/ubuntu/wav2vec2/singlish-test/2716.wav',
  'melissa passed her number to the young man'),
 ('/home/ubuntu/wav2

In [7]:
import kenlm
from pyctcdecode.language_model import LanguageModel

In [8]:
lm = malaya_speech.stt.language_model(model = 'bahasa-manglish-combined')

In [9]:
kenlm_model = kenlm.Model(lm)
language_model = LanguageModel(kenlm_model, alpha = 0.01, beta = 0.5)

In [10]:
y, _ = malaya_speech.load(test_set[0][0])
y1, _ = malaya_speech.load(test_set[1][0])

In [11]:
model = malaya_speech.stt.deep_transducer(model = 'conformer-stack-2mixed')

2022-06-11 22:39:42.006841: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-11 22:39:42.010631: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-11 22:39:42.010647: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-11 22:39:42.010651: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-06-11 22:39:42.010687: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-06-11 22:39:42.010709: I

In [12]:
%%time

model.beam_decoder_lm([y, y1], language_model)

CPU times: user 8 s, sys: 354 ms, total: 8.35 s
Wall time: 4.97 s


['kenapa orang orang muslim tidak menonjol melawan sepanyol',
 'and as if there was monday enough boiled decided to pick it up']

In [13]:
def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [14]:
from tqdm import tqdm

wer, cer = [], []
wer_lm, cer_lm = [], []

batch_size = 4
for i in tqdm(range(0, len(audio), batch_size)):
    
    batch_x = audio[i: i + batch_size]
    batch_y = labels[i: i + batch_size]
    
    ys = [malaya_speech.load(b)[0] for b in batch_x]
    pred = model.beam_decoder(ys)
    pred_lm = model.beam_decoder_lm(ys, language_model)
    
    for k in range(len(pred)):
        
        wer.append(calculate_wer(batch_y[k], pred[k]))
        cer.append(calculate_cer(batch_y[k], pred[k]))
        
        wer_lm.append(calculate_wer(batch_y[k], pred_lm[k]))
        cer_lm.append(calculate_cer(batch_y[k], pred_lm[k]))

100%|██████████| 1086/1086 [2:59:11<00:00,  9.90s/it] 


In [15]:
import numpy as np

np.mean(wer), np.mean(cer), np.mean(wer_lm), np.mean(cer_lm)

(0.10360847178845402,
 0.05006960208570647,
 0.10291117410644485,
 0.05020132668208229)

In [16]:
index_malay = [no for no, i in enumerate(audio[: len(wer)]) if 'malay-test/' in i]
index_singlish = [no for no, i in enumerate(audio[: len(wer)]) if 'singlish-test/' in i]

In [17]:
np.mean(np.array(wer)[index_malay]), np.mean(np.array(cer)[index_malay]), np.mean(np.array(wer_lm)[index_malay]), np.mean(np.array(cer_lm)[index_malay])

(0.18898839540996407,
 0.07268455314935462,
 0.18594507707252805,
 0.07514044348046696)

In [18]:
np.mean(np.array(wer)[index_singlish]), np.mean(np.array(cer)[index_singlish]), np.mean(np.array(wer_lm)[index_singlish]), np.mean(np.array(cer_lm)[index_singlish])

(0.0853587814921547,
 0.04523572738224437,
 0.08516293835091154,
 0.04487066885845439)